## Minimal working example

Script creation for the LDclumping.ipynb

In [88]:
%save MWE_ldclump.sh -f

# Defining bash variables for the different paths,

output=output
bfile=~/MWE0612/genotypes.bed
bgenFile=`echo ~/MWE0612/imputed_genotypes_chr2?.bgen`
sampleFile=~/MWE0612/imputed_genotypes.sample
sumstatsFile=~/MWE0612/output/phenotypes_BMI.fastGWA.snp_stats.gz
unrelated_samples=~/MWE0612/unrelated_samples.txt
ld_sample_size=100
clump_field=P
clump_p1=0.05
clump_p2=1
clump_r2=0.2
clump_kb=1000
clump_annotate=OR
numThreads=20
job_size=1

#Running the workflow for BMI trait

sos dryrun ~/UKBB_GWAS_DEV/workflow/LD_Clumping.ipynb \
    --cwd $output \
    --bfile $bfile \
    --bgenFile $bgenFile \
    --sampleFile $sampleFile \
    --sumstatsFile $sumstatsFile \
    --unrelated_samples $unrelated_samples \
    --ld_sample_size $ld_sample_size \
    --clump_field $clump_field \
    --clump_p1 $clump_p1 \
    --clump_p2 $clump_p2 \
    --clump_r2 $clump_r2 \
    --clump_kb $clump_kb \
    --clump_annotate $clump_annotate \
    --numThreads $numThreads \
    --job_size $job_size
    -q localhost \
    -s build &> 062420-sos-LD-MWE.log

Cell content saved to MWE_ldclump.sh, use option -r to also execute the cell.

In [1]:
cd

In [ ]:
sos run ~/project/bioworkflows/GWAS/LMM.ipynb regenie\
    --cwd /home/dc2325/output \
    --bfile /home/dc2325/data/genotypes.bed \
    --sampleFile /home/dc2325/data/imputed_genotypes.sample \
    --bgenFile /home/dc2325/data/imputed_genotypes_chr*.bgen \
    --phenoFile /home/dc2325/data/phenotypes.txt \
    --formatFile /home/dc2325/data/regenie_template.yml \
    --phenoCol BMI \
    --covarCol SEX \
    --qCovarCol AGE \
    --numThreads 8 \
    --bsize 1000 \
    --lowmem /home/dc2325/output \
    --trait \
    --minMAC 4 \
    --bgenMinMAF 0.05 \
    --bgenMinINFO 0.8 \
    --reverse_log_p 

INFO: Running regenie_1: Run REGENIE step 1: fitting the null


In [ ]:
regenie \
  --step 1 \
  --bed example/example \
  --exclude example/snplist_rm.txt \
  --covarFile example/covariates.txt \
  --phenoFile example/phenotype.txt \
  --remove example/fid_iid_to_remove.txt \
  --bsize 100 \
  --lowmem \
  --lowmem-prefix tmp_rg \
  --out fit_quant_out

In [18]:
    regenie \
     --step 2 \
     --bgen /home/dc2325/data/imputed_genotypes_chr*.bgen \
     --keep /home/dc2325/output/cache/genotypes.regenie.keep\
     --phenoFile /home/dc2325/output/phenotypes.regenie_phenotype \
     --covarFile /home/dc2325/output/phenotypes.regenie_covar\
     --phenoColList BMI \
     --covarColList AGE,SEX \
     --firth 0.01 --approx \
     --pred /home/dc2325/output/phenotypes_BMI.regenie_pred.list \
     --bsize 400 \
     --minMAC 4 \
     --split \
     --threads 5 \
     --out test_BMI

Start time: Tue Aug 11 10:35:59 2020

              |==================================|
              |           REGENIE v1.0.5.5       |
              |==================================|

Copyright (c) 2020 Joelle Mbatchou and Jonathan Marchini.
Distributed under the MIT License.

Log of output saved in file : test_BMI.log

Options in effect:
  --step 2 \
  --bgen /home/dc2325/data/imputed_genotypes_chr21.bgen \
  --keep /home/dc2325/output/cache/genotypes.regenie.keep \
  --phenoFile /home/dc2325/output/phenotypes.regenie_phenotype \
  --covarFile /home/dc2325/output/phenotypes.regenie_covar \
  --phenoColList BMI \
  --covarColList AGE,SEX \
  --firth \
  --approx \
  --pred /home/dc2325/output/phenotypes_BMI.regenie_pred.list \
  --bsize 400 \
  --minMAC 4 \
  --split \
  --threads 5 \
  --out test_BMI

Association testing mode with multithreading using OpenMP
 * bgen             : [/home/dc2325/data/imputed_genotypes_chr21.bgen]
   -summary : bgen file (v1.2 layout, compressed)

In [30]:
cd ~/project/UKBB_GWAS_DEV/output

In [31]:
# Common variables
tpl_file=~/project/UKBB_GWAS_DEV/farnam.yml
bfile=~/data/genotypes.bed
sampleFile=~/data/imputed_genotypes.sample
bgenFile=~/data/imputed_genotypes_chr*.bgen
# LMM directories
lmm_dir_regenie=~/output
lmm_sos=~/project/bioworkflows/GWAS/LMM.ipynb
lmm_sbatch_regenie=../output/$(date +"%Y-%m-%d")_mwe_regenie.sbatch
phenoFile=~/data/phenotypes.txt
lowmem=~/output
## LMM variables 
phenoCol=BMI
covarCol=SEX
covarMaxLevels=10
qCovarCol=AGE
numThreads=8
bgenMinMAF=0.05
bgenMinINFO=0.8
minMAC=4
lmm_job_size=1
bsize=1000
ylim=0

In [32]:
lmm_args="""regenie
    --cwd $lmm_dir_regenie 
    --bfile $bfile 
    --sampleFile $sampleFile
    --bgenFile $bgenFile 
    --phenoFile $phenoFile  
    --phenoCol $phenoCol 
    --covarCol $covarCol 
    --qCovarCol $qCovarCol
    --bgenMinMAF $bgenMinMAF
    --bgenMinINFO $bgenMinINFO
    --bsize $bsize
    --minMAC $minMAC
    --lowmem $lowmem
    --numThreads $numThreads  
    --job_size $lmm_job_size
    --ylim $ylim
    --trait
"""

sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_regenie \
    --args "$lmm_args"

INFO: Running farnam: Configuration for Yale `farnam` cluster
INFO: farnam is completed.
INFO: farnam output:   ../output/2020-08-10_mwe_regenie.sbatch
INFO: Workflow farnam (ID=280055fda8fd6626) is executed successfully with 1 completed step.
